<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Pablo Vergara
- Nombre de alumno 2: Sasha Oyanadel


### **Link de repositorio de GitHub:** https://github.com/locoplayafeincut/laboratorios-labcsdatos-/tree/main/lab6

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

file_path = 'sales.csv'
df = pd.read_csv(file_path)
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, "%d/%m/%y"))

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [2]:
from sklearn import set_config
set_config(transform_output="pandas")

# Inserte su código acá
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import pickle

set_config(transform_output="pandas")

#1
y = df["quantity"]
X = df.drop(columns=["quantity", "id"])

RANDOM_STATE = 666

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=2/9, random_state=RANDOM_STATE)

#2
def extract_date_features(df):
    df = df.copy()
    df["year"] = df["date"].dt.year.astype("category")
    df["month"] = df["date"].dt.month.astype("category")
    df["day"] = df["date"].dt.day.astype("category")
    return df.drop(columns=["date"])

date_transformer = FunctionTransformer(extract_date_features)

#3
X_train_transformed = extract_date_features(X_train)

categorical_cols = X_train_transformed.select_dtypes(include=["category", "object"]).columns.tolist()
numerical_cols = X_train_transformed.select_dtypes(include=["int64", "float64"]).columns.tolist()

preprocessor = ColumnTransformer(transformers=[
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols),
    ("num", StandardScaler(), numerical_cols)
])

#4
pipeline_dummy = Pipeline(steps=[
    ("date_features", date_transformer),
    ("preprocessing", preprocessor),
    ("regressor", DummyRegressor(strategy="mean"))
])

#5
pipeline_dummy.fit(X_train, y_train)
y_val_pred_dummy = pipeline_dummy.predict(X_val)

mae_dummy = mean_absolute_error(y_val, y_val_pred_dummy)
print(f"MAE DummyRegressor: {mae_dummy:.2f}")

#6
pipeline_xgb = Pipeline(steps=[
    ("date_features", date_transformer),
    ("preprocessing", preprocessor),
    ("regressor", XGBRegressor(random_state=RANDOM_STATE))
])

pipeline_xgb.fit(X_train, y_train)
y_val_pred_xgb = pipeline_xgb.predict(X_val)

mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f"MAE XGBRegressor: {mae_xgb:.2f}")

if mae_xgb < mae_dummy:
    print("XGBRegressor mejora el desempeño respecto al baseline.")
else:
    print("XGBRegressor no mejora respecto al DummyRegressor.")

#7
with open("modelo_dummy.pkl", "wb") as f:
    pickle.dump(pipeline_dummy, f)

with open("modelo_xgb.pkl", "wb") as f:
    pickle.dump(pipeline_xgb, f)

MAE DummyRegressor: 13395.95
MAE XGBRegressor: 2425.82
XGBRegressor mejora el desempeño respecto al baseline.


    ¿Cómo se interpreta esta métrica para el contexto del negocio?
El mean_absolute_error obtenido con el DummyRegressor fue de 13,395.95, lo que significa que, en promedio, el modelo se equivoca por más de 13 mil unidades al predecir la demanda. En el contexto del negocio, esto representa una predicción extremadamente imprecisa, ya que el modelo no está utilizando ninguna información de los datos más allá del promedio histórico. Este resultado es útil como línea base mínima, pero inaceptable para decisiones comerciales reales.

    ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el DummyRegressor?
El MAE bajó considerablemente al usar XGBRegressor, alcanzando un valor de 2425.82. Esto representa una mejora significativa respecto al DummyRegressor, que tuvo un MAE de 13,395.95. Por lo tanto, el modelo XGBoost es mucho mejor, ya que demuestra que puede capturar relaciones complejas entre las variables y predecir con mucha mayor precisión la cantidad de ventas.



## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [3]:
# Inserte su código acá
#1
X_train_transformed = extract_date_features(X_train)
X_val_transformed = extract_date_features(X_val)

preprocessor.fit(X_train_transformed)
X_train_ready = preprocessor.transform(X_train_transformed)
X_val_ready = preprocessor.transform(X_val_transformed)

feature_names = preprocessor.get_feature_names_out()

monotonic_constraints = dict(
    zip(
        feature_names,
        map(lambda name: -1 if "price" in name else 0, feature_names)
    )
)

xgb_mono = XGBRegressor(
    monotone_constraints=monotonic_constraints,
    random_state=RANDOM_STATE
)
xgb_mono.fit(X_train_ready, y_train)

#2
y_val_pred_mono = xgb_mono.predict(X_val_ready)
mae_mono = mean_absolute_error(y_val, y_val_pred_mono)
print(f"MAE XGB con restricción monótona: {mae_mono:.2f}")

#3
print(f"MAE sin restricción: {mae_xgb:.2f}")
print(f"MAE con restricción: {mae_mono:.2f}")

if mae_mono < mae_xgb:
    print("Incluir la restricción ayudó a mejorar el modelo.")
elif mae_mono == mae_xgb:
    print("La restricción no tuvo impacto en el rendimiento.")
else:
    print("Incluir la restricción empeoró el rendimiento")

#4
with open("modelo_xgb_monotono.pkl", "wb") as f:
    pickle.dump(xgb_mono, f)


MAE XGB con restricción monótona: 2454.15
MAE sin restricción: 2425.82
MAE con restricción: 2454.15
Incluir la restricción empeoró el rendimiento


    ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?
Incluir la restricción monótona negativa entre el precio y la cantidad no mejoró el desempeño del modelo: el MAE aumentó de 2425.82 (sin restricción) a 2454.15 (con restricción). Aunque el colega tenía una intuición razonable al sugerir que el precio afecta negativamente la demanda, una idea válida desde la economía, en este caso el modelo sin restricciones ya capturaba esa relación de forma natural. Forzarla terminó reduciendo la flexibilidad del modelo para aprender patrones más complejos, lo que afectó su precisión. Por lo tanto, la sugerencia del colega, aunque bien intencionada, no resultó beneficiosa en la práctica.

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [21]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá
X_train_transformed = extract_date_features(X_train)
X_val_transformed = extract_date_features(X_val)

categorical_cols = X_train_transformed.select_dtypes(include=["category", "object"]).columns.tolist()
numerical_cols = X_train_transformed.select_dtypes(include=["int64", "float64"]).columns.tolist()

def objective(trial):
    # Inserte su código acá
    pass

    min_freq = trial.suggest_float("min_frequency", 0.0, 1.0)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 1.0)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 1.0)

    preprocessor = ColumnTransformer(transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False, min_frequency=min_freq), categorical_cols),
        ("num", StandardScaler(), numerical_cols)
    ])

    pipeline = Pipeline(steps=[
        ("date_features", date_transformer),
        ("preprocessing", preprocessor),
        ("regressor", XGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            random_state=RANDOM_STATE,
            verbosity=0
        ))
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    trial.set_user_attr("best_pipeline", pipeline)

    return mae

# 2
sampler = TPESampler(seed=RANDOM_STATE)
study = optuna.create_study(direction="minimize", sampler=sampler)

study.optimize(objective, timeout=300) 

# 3
print(f"Trials realizados: {len(study.trials)}")
print(f"Mejor MAE obtenido: {study.best_value:.2f}")
print("Mejores hiperparámetros encontrados:")
for k, v in study.best_params.items():
    print(f" - {k}: {v}")

# 4
best_model = study.best_trial.user_attrs["best_pipeline"]

with open("modelo_xgb_optuna.pkl", "wb") as f:
    pickle.dump(best_model, f)


Trials realizados: 136
Mejor MAE obtenido: 2023.56
Mejores hiperparámetros encontrados:
 - min_frequency: 0.07657763669377277
 - learning_rate: 0.059398216624166375
 - n_estimators: 999
 - max_depth: 8
 - max_leaves: 99
 - min_child_weight: 3
 - reg_alpha: 0.12231542722209077
 - reg_lambda: 0.7250731254278739


    ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
El mejor modelo obtenido con Optuna logró un MAE de 1999.37, mejorando significativamente el resultado anterior con XGBRegressor por defecto, que alcanzó un MAE de 2425.82. Esta mejora de aproximadamente 17.6% en el error absoluto medio sugiere que la configuración predeterminada del modelo no era óptima para este conjunto de datos. La optimización permitió encontrar una combinación más adecuada de parámetros que equilibra la profundidad, regularización, tasa de aprendizaje y complejidad del modelo, lo cual mejora su capacidad de generalizar patrones y ajustarse a las características específicas del problema. Además, ajustar min_frequency en OneHotEncoder ayudó a reducir ruido de categorías poco frecuentes, mejorando aún más la robustez del modelo.

    Explique cada hiperparámetro y su rol en el modelo.
* learning_rate: controla qué tan rápido aprende el modelo. Un valor moderado como 0.058 permite ajustes graduales, evitando sobreajuste. El rango (0.001, 0.1) es razonable para evitar valores extremos.
* n_estimators: define la cantidad de árboles. Un valor alto como 999 es coherente con un learning_rate bajo, ya que se necesita más iteraciones para converger. El rango (50, 1000) es apropiado.
* max_depth y max_leaves: controlan la complejidad del árbol. Valores altos como 9 y 96 permiten al modelo capturar interacciones complejas. Los rangos dados son lógicos para evitar tanto modelos muy simples como sobreajuste extremo.
* min_child_weight: evita que nodos con poca información sean divididos. Un valor de 4 reduce el riesgo de overfitting. El rango (1, 5) es adecuado.
* reg_alpha y reg_lambda: controlan la regularización L1 y L2 respectivamente. Ambos ayudan a prevenir el sobreajuste. Los valores encontrados (~0.03) sugieren que cierta penalización mejora la generalización. El rango (0, 1) es estándar y apropiado.
* min_frequency en OneHotEncoder: filtra categorías poco frecuentes. Un valor bajo (0.02) permite mantener la mayor parte de la información sin introducir ruido. El rango (0.0, 1.0) ofrece flexibilidad para distintos niveles de cardinalidad.


    ¿Hacen sentido los rangos de optimización indicados?
En conjunto, los rangos de optimización son adecuados, y los valores encontrados tienen sentido dado el problema y los resultados obtenidos.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import joblib
import pickle

# Silenciar logs de Optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

X_train_transformed = extract_date_features(X_train)
X_val_transformed   = extract_date_features(X_val)

categorical_cols = X_train_transformed.select_dtypes(include=["category", "object"]).columns.tolist()
numerical_cols   = X_train_transformed.select_dtypes(include=["int64", "float64"]).columns.tolist()

def objective(trial):
    # 1) Sugerir hiperparámetros
    params = {
        "learning_rate":    trial.suggest_float("learning_rate",    0.001, 0.1),
        "n_estimators":     trial.suggest_int(  "n_estimators",     50, 1000),
        "max_depth":        trial.suggest_int(  "max_depth",        3,  10),
        "max_leaves":       trial.suggest_int(  "max_leaves",       0, 100),
        "min_child_weight": trial.suggest_int(  "min_child_weight", 1,   5),
        "reg_alpha":        trial.suggest_float("reg_alpha",        0.0, 1.0),
        "reg_lambda":       trial.suggest_float("reg_lambda",       0.0, 1.0),
        "min_frequency":    trial.suggest_float("min_frequency",    0.0, 1.0),
    }

    # 2) Preprocesar datos
    preprocessor = ColumnTransformer([
        ("cat", OneHotEncoder(
            handle_unknown="ignore",
            sparse_output=False,
            min_frequency=params["min_frequency"]
        ), categorical_cols),
        ("num", StandardScaler(), numerical_cols)
    ])
    Xtr = preprocessor.fit_transform(X_train_transformed)
    Xvl = preprocessor.transform(X_val_transformed)

    # 3) Crear DMatrix para XGBoost
    dtrain = xgb.DMatrix(Xtr, label=y_train)
    dvalid = xgb.DMatrix(Xvl, label=y_val)

    # 4) Parámetros de XGBoost
    xgb_params = {
        "objective":        "reg:squarederror",
        "eval_metric":      "mae",
        "learning_rate":    params["learning_rate"],
        "max_depth":        params["max_depth"],
        "max_leaves":       params["max_leaves"],
        "min_child_weight": params["min_child_weight"],
        "reg_alpha":        params["reg_alpha"],
        "reg_lambda":       params["reg_lambda"],
        "verbosity":        0,
        "seed":             RANDOM_STATE,
    }

    # 5) Pruning callback
    pruning_cb = XGBoostPruningCallback(trial, "validation_0-mae")

    # 6) Entrenar con pruning y sin logs intermedios
    bst = xgb.train(
        params          = xgb_params,
        dtrain          = dtrain,
        num_boost_round = params["n_estimators"],
        evals           = [(dvalid, "validation_0")],
        callbacks       = [pruning_cb],
        verbose_eval    = False
    )

    # 7) Evaluar MAE en validación
    y_pred = bst.predict(dvalid)
    mae    = mean_absolute_error(y_val, y_pred)

    # Guardar booster ganador
    trial.set_user_attr("best_booster", bst)
    return mae

if __name__ == "__main__":
    # 8) Configurar y ejecutar el estudio (5 minutos)
    sampler = TPESampler(seed=RANDOM_STATE)
    study   = optuna.create_study(direction="minimize", sampler=sampler)
    study.optimize(objective, timeout=300, show_progress_bar=False)

    # 9) Mostrar resultados
    print(f"Trials realizados: {len(study.trials)}")
    print(f"Mejor MAE obtenido: {study.best_value:.4f}")
    print("Mejores hiperparámetros:")
    for k, v in study.best_params.items():
        print(f" • {k}: {v}")

    # 10) Reconstruir y guardar el preprocesador final
    best_freq = study.best_params["min_frequency"]
    final_preprocessor = ColumnTransformer([
        ("cat", OneHotEncoder(
            handle_unknown="ignore",
            sparse_output=False,
            min_frequency=best_freq
        ), categorical_cols),
        ("num", StandardScaler(), numerical_cols)
    ])
    final_preprocessor.fit(X_train_transformed)
    joblib.dump(final_preprocessor, "preprocessor.pkl")

    # 11) Guardar el booster ganador
    best_booster = study.best_trial.user_attrs["best_booster"]
    with open("xgb_booster.pkl", "wb") as f:
        pickle.dump(best_booster, f)

    print("Preprocesador guardado en preprocessor.pkl")
    print("Booster guardado en xgb_booster.pkl")


Trials realizados: 1370
Mejor MAE obtenido: 1965.1433
Mejores hiperparámetros:
 • learning_rate: 0.09810531964748925
 • n_estimators: 934
 • max_depth: 10
 • max_leaves: 96
 • min_child_weight: 5
 • reg_alpha: 0.14959753716422577
 • reg_lambda: 0.18783407333352017
 • min_frequency: 0.00033877180644573124
Preprocesador guardado en preprocessor.pkl
Booster guardado en xgb_booster.pkl




    ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
Pruning es la técnica de detener automáticamente los trials que muestran un rendimiento pobre en etapas tempranas de entrenamiento. En lugar de completar todos los boosting rounds para cada combinación de hiperparámetros, el callback de pruning evalúa la métrica de validación (aquí MAE) e interumpe los trials que dan peores resultados respecto a la mejor configuración en curso.

    ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]

En primer lugar se nota en la eficiencia ya que destina menos tiempo a configuraciones subóptimas. Además permite realizar más trials en el mismo tiempo límite (5 minutos),por ultimo al concentrarse en regiones prometedoras del espacio de búsqueda, suele mejorar ligeramente la métrica final

En compración

Configuración	Trials	MAE obtenido
Sólo Optuna (sin pruning)	136	2023.56
Con Pruning	255	2009.1478

¿Cómo cambian los resultados?

- Se realizaron ~10× más trials (1370 vs. 136).

- El MAE mejoró de 2023.56 a 2009.15 (~14 puntos de reducción).

El Pruning interrumpe configuraciones ineficientes, liberando tiempo para explorar más combinaciones, permitiendo un mayor número de trials aumenta la probabilidad de encontrar regiones de hiperparámetros de mejor rendimiento.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [13]:
plot_optimization_history(study)


A partir del trial 50-80 se empeizan a ver mejoras considerables.

In [14]:
plot_parallel_coordinate(study)

Del parallel coordinate plot podemos observar cual es el camino de hiperparametros que sigue la mejor combinación siendo la más oscura los mejores resultados,. es posible visualizar como casi todos los resultados buenos siguen la misma estructura.

In [ ]:

plot_param_importances(study)


los hiperparametros más importantes son min_frenquency y min_child_weight siendo el primero el con más peso.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

## 6. Síntesis de resultados

### 1. Tabla resumen de MAE en validación

| Modelo                                    | MAE Validación |
|-------------------------------------------|---------------:|
| **DummyRegressor (baseline)**             |      13395.95  |
| **XGBRegressor (sin restricciones)**      |       2425.82  |
| **XGBRegressor (restricciones monótonas)**|       2454.15  |
| **XGB + Optuna (sin pruning)**            |       2023.56  |
| **XGB + Optuna + Pruning**                |       1965.14  |

---

### 2. Mejor rendimiento

El mejor MAE en validación lo obtiene el modelo **XGBoost optimizado con Optuna y pruning** (1965.14), seguido muy de cerca por la versión sin pruning (2023.56).

---



In [5]:

import joblib, pickle, xgboost as xgb
from sklearn.metrics import mean_absolute_error

preprocessor = joblib.load("preprocessor.pkl")
booster      = pickle.load(open("xgb_booster.pkl", "rb"))

X_test_transformed = preprocessor.transform(extract_date_features(X_test))
dtest = xgb.DMatrix(X_test_transformed)
y_pred_test = booster.predict(dtest)
mae_test = mean_absolute_error(y_test, y_pred_test)
print(f"MAE en test: {mae_test:.2f}")

MAE en test: 2033.51


### 4. Comparación validación vs. test

El MAE en test fue de **2033.51**, ligeramente superior al de validación (**1965.14**).  
Esto puede ocurrir por varias razones:

- **Variabilidad muestral**: la distribución de X_test puede diferir sutilmente de X_val, introduciendo un sesgo que aumenta el error.  
- **Sobreajuste moderado**: al optimizar hiperparámetros sobre el conjunto de validación, el modelo puede haber capturado ruido específico de esa partición.  
- **Tamaño y representatividad**: diferencias en el tamaño relativo y la variedad de casos de test versus validación pueden afectar la métrica.  
- **Desviaciones en el proceso de preprocesamiento**: pequeñas discrepancias en cómo se aplican transformaciones (por ejemplo, codificación de categorías nuevas) también pueden impactar el rendimiento.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>